<a href="https://colab.research.google.com/github/Aarman-J/Play-Generator/blob/main/Play_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np


Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


In [ ]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 [==============================] - 0s 0us/step


In [ ]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
len(text)

1115394

In [ ]:
vocab = sorted(set(text))
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)
def text_to_int(text):
  return np.array([char2idx[c] for c in text])
text_to_int(text)
text_as_int = text_to_int(text)

In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy()
  except:
    pass
  return ''.join(idx2char[ints])

print(text_as_int)

[18 47 56 ... 45  8  0]


In [ ]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)


In [ ]:
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry


In [ ]:
for x, y in dataset.take(2):
  print(int_to_text(x))
  print(int_to_text(y))

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
irst Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You 
are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you 
re all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you k


In [ ]:
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)


In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           16640     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 dense (Dense)               (64, None, 65)            66625     
                                                                 
Total params: 5,330,241
Trainable params: 5,330,241
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [ ]:
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[ 5.0606476e-03  5.2575115e-03 -2.4749697e-03 ... -1.6662343e-04
   -4.1138974e-04  7.4139098e-03]
  [ 2.8151390e-03  2.8036321e-03 -4.5163347e-03 ... -2.5278986e-03
   -4.7366438e-03  5.9787850e-03]
  [ 9.5914947e-03  6.0152821e-03 -3.3542323e-03 ... -8.2857050e-03
   -3.4505224e-03  1.5160327e-03]
  ...
  [-6.1307461e-03 -7.8319106e-03 -8.3166603e-03 ...  1.2931371e-02
    3.3414273e-03 -4.7813943e-03]
  [ 5.6294864e-04 -1.8021325e-05 -6.6970689e-03 ...  3.7074478e-03
    1.2464256e-03 -4.9585216e-03]
  [-4.5475727e-03  1.1958978e-03 -1.6721450e-03 ... -2.0427662e-03
   -2.4684207e-03 -5.7352809e-03]]

 [[ 4.3263967e-04 -1.9867599e-03  4.2449878e-04 ... -3.5796838e-04
    3.7634559e-04 -8.2952296e-04]
  [-1.3915010e-03 -4.9399035e-03  6.7923189e-04 ... -6.6297839e-04
   -7.2822901e-03  3.0900484e-03]
  [ 6.1644986e-03 -7.0158765e-04 -8.2603563e-04 ... -6.2114219e-03
   -6.8133976e-03 -9.6974400e-04]
  ...
  [ 4.0959087e-03  1.4250783e-03 -3.0982541e-03 ... -2.1803491e

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d arra

100
tf.Tensor(
[[ 5.0606476e-03  5.2575115e-03 -2.4749697e-03 ... -1.6662343e-04
  -4.1138974e-04  7.4139098e-03]
 [ 2.8151390e-03  2.8036321e-03 -4.5163347e-03 ... -2.5278986e-03
  -4.7366438e-03  5.9787850e-03]
 [ 9.5914947e-03  6.0152821e-03 -3.3542323e-03 ... -8.2857050e-03
  -3.4505224e-03  1.5160327e-03]
 ...
 [-6.1307461e-03 -7.8319106e-03 -8.3166603e-03 ...  1.2931371e-02
   3.3414273e-03 -4.7813943e-03]
 [ 5.6294864e-04 -1.8021325e-05 -6.6970689e-03 ...  3.7074478e-03
   1.2464256e-03 -4.9585216e-03]
 [-4.5475727e-03  1.1958978e-03 -1.6721450e-03 ... -2.0427662e-03
  -2.4684207e-03 -5.7352809e-03]], shape=(100, 65), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values re

65
tf.Tensor(
[ 5.0606476e-03  5.2575115e-03 -2.4749697e-03 -3.1291253e-03
  3.5761718e-03 -6.8166014e-04 -3.9709392e-03 -1.5892304e-03
 -8.2123053e-04  1.4901401e-03 -1.7352455e-03 -7.3072151e-05
  4.4408706e-03 -3.4606177e-04  5.8682119e-03  2.6806816e-04
  5.4035531e-03 -4.8550370e-04  6.7099640e-03  5.6486297e-03
 -1.5628530e-03 -8.4639769e-03 -8.7481181e-05  1.2839552e-03
 -5.3569563e-03  2.0316096e-03 -4.1162875e-03 -4.7896802e-04
 -3.7107607e-03  3.2348982e-03  5.8410149e-03 -3.7090804e-03
  6.9899706e-04  6.5741139e-03  7.4370950e-04 -4.6760226e-03
  7.1285450e-04  1.5196821e-03  1.0644258e-03 -1.3804120e-03
 -4.8964069e-04 -2.0742305e-03 -1.0622925e-03 -3.0528412e-03
  1.8668065e-03 -6.2716956e-04 -1.4739905e-03  1.9937204e-03
 -8.8548171e-04  4.8111589e-04  6.1415751e-03  4.5833332e-03
 -3.1908578e-03 -4.8399023e-03 -8.3953934e-04 -4.3993136e-03
  6.6292542e-03  4.6579866e-03 -3.7597748e-04  1.3731016e-03
 -2.5514788e-03 -2.9269736e-03 -1.6662343e-04 -4.1138974e-04
  7.413909

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"McPkX&mIEro:nxjZvL'c.:Gg lHRsikTAoLoUdB.HSXPEfMS;asRph!i aEFDLFKGdJpwj-U-;!MCZWGKpD$:?QmnaVpojc;yAjh"

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
model.compile(optimizer='adam', loss=loss)
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=50, callbacks=[checkpoint_callback])

Epoch 1/50
172/172 [==============================] - 17s 71ms/step - loss: 2.5954
Epoch 2/50
172/172 [==============================] - 13s 66ms/step - loss: 1.9089
Epoch 3/50
172/172 [==============================] - 15s 67ms/step - loss: 1.6541
Epoch 4/50
172/172 [==============================] - 13s 67ms/step - loss: 1.5160
Epoch 5/50
172/172 [==============================] - 13s 67ms/step - loss: 1.4314
Epoch 6/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3735
Epoch 7/50
172/172 [==============================] - 13s 67ms/step - loss: 1.3254
Epoch 8/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2872
Epoch 9/50
172/172 [==============================] - 13s 68ms/step - loss: 1.2513
Epoch 10/50
172/172 [==============================] - 14s 68ms/step - loss: 1.2176
Epoch 11/50
172/172 [==============================] - 13s 68ms/step - loss: 1.1824
Epoch 12/50
172/172 [==============================] - 14s 69ms/step - loss: 1.1471
E

In [ ]:
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 8000

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension

      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: the dog died
the dog died,
And he mismalce shall not with me to-morrow
And haply suitors have wed foreral,
Before so young a tace, he's and eyed with being sleep,
Unvil an exchmons.

BRUTUS:
Mancau shoot.

QUEEN ELIZABETH:
O thou, sirrah, indeed, sir, the
tarting of your worthy rage,
Husband in oblivion. You are plebeix,
That cease that thou, whosood enr,
To lock upon partial sides, contrim ind Gaunt.

DUKE VINCENTIO:
Not I, believe me: I have found the wind;
Where he are ears with me untined fault, and so disgraced
put on the basiness, who back to make thee offer,
At Capulet; and when we are
Shows her for ever. He should have none before him.

ISABELLA:
I will about to make you gainst Perchis publicly:
how he let's do, though e the most melanch is well.
Welcome, he hasbrace:
I am no further like a napiliance of our garliage.

MENENIUS:
That's as much below,
And with his virtue space I have in vain as spen.
I never yet been st, ought show him; he must die tomorr